### Prompt: 

For every mountain in the Tour de FiveThirtyEight, the first few riders to reach the summit are awarded points. The rider with the most such points at the end of the Tour is named “King of the Mountains” and gets to wear a special polka dot jersey.

At the moment, you are racing against three other riders up one of the mountains. The first rider over the top gets 5 points, the second rider gets 3, the third rider gets 2, and the fourth rider gets 1.

All four of you are of equal ability — that is, under normal circumstances, you all have an equal chance of reaching the summit first. But there’s a catch — two of your competitors are on the same team. Teammates are able to work together, drafting and setting a tempo up the mountain. Whichever teammate happens to be slower on the climb will get a boost from their faster teammate, and the two of them will both reach the summit at the faster teammate’s time.

As a lone rider, the odds may be stacked against you. In your quest for the polka dot jersey, how many points can you expect to win on this mountain, on average?

### Assumptions:

- Every player has an equal likelihood of winning of `p` at anytime, so `p=0.25`
- P3 & P4 will always be sequential (e.g. 1st/2nd, 2nd/3rd, 3rd/4th)
- We only care about P1 final values

In [6]:
import numpy as np
import statistics
import time 

# track time
start = time.time()

player_1_points = []

for _ in range(5000000):
    
    # determining the first place person
    first = np.random.choice(np.arange(1, 5), p=[0.25, 0.25, 0.25, 0.25])

    # if P3 is first, then we know P4 has to be second 
    # if P4 is first, then we know P3 has to be second
    if first == 3 or first == 4:
        
        # randomly determine between P1 & p3 who is third
        third =  np.random.choice(np.arange(1, 3), p=[0.5, 0.5])
        
        if third == 1:
            player_1_points.append(2) #p1 was 3rd, so need to add 2 points
        else:
            player_1_points.append(1) #p1 was not 3rd, so 4th by default 
 
    # if P2 is first, then next 3 spots are up for grabs 
    elif first == 2:
        
        # randomly choose 2nd, 3rd and 4th place 
        second =  np.random.choice([1,3,4], p=[1/3, 1/3,1/3])


        # if P3 or P4 is second, then P1 automatically 4th 
        if second == 3 or second == 4:
            player_1_points.append(1) # p1 automatically in 4th 

        else:
            player_1_points.append(3) # p1 is second by default 
            
    # p1 is first
    else:
        player_1_points.append(5)

len(player_1_points)

mean_score = statistics.mean(player_1_points) # mean over n runs

print(mean_score)

end = time.time()

print(f"Total time was {end - start}")

2.4169622
Total time was 239.76346635818481


### A More Efficient Solve

Attempting to speed attempt up

In [17]:
from numba import jit


@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def ride_sim(): # Function is compiled to machine code when called the first time
    
    # determining the first place person
    first = np.random.choice(np.arange(1, 5))

    if first == 3 or first == 4:
        third =  np.random.choice(np.arange(1, 3))
        if third == 1:
            return 2 #p1 was 3rd, so need to add 2 points
        else:
            return 1 #p1 was not 3rd, so 4th by default 
 
    elif first == 2:
        second =  np.random.choice([1,3,4])
        if second == 3 or second == 4:
            return 1 # p1 automatically in 4th 
        else:
            return 3 # p1 is second by default 
    else:
        return 5

In [18]:
# track time for numba 
start = time.time()
sims = 5
player_1_points = np.zeros(shape=(sims,1))

for _ in range(sims):
    np.append(player_1_points, ride_sim())
    
# get stats
mean_score = statistics.mean(player_1_points) # mean over n runs
print(mean_score)
end = time.time()

print(f"Total time was {end - start}")

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in method choice of numpy.random.mtrand.RandomState object at 0x0000017A7E8A8E40>) found for signature:
 
 >>> choice(list(int64)<iv=[1, 3, 4]>)
 
There are 2 candidate implementations:
[1m      - Of which 1 did not match due to:
      Overload in function 'choice': File: numba\cpython\randomimpl.py: Line 1346.
        With argument(s): '(list(int64)<iv=None>)':[0m
[1m       Rejected as the implementation raised a specific error:
         TypeError: np.random.choice() first argument should be int or array, got list(int64)<iv=None>[0m
  raised from C:\Users\David\AppData\Local\Programs\Python\Python38\lib\site-packages\numba\cpython\randomimpl.py:1383
[1m      - Of which 1 did not match due to:
      Overload in function 'choice': File: numba\cpython\randomimpl.py: Line 1346.
        With argument(s): '(list(int64)<iv=[1, 3, 4]>)':[0m
[1m       Rejecte